## Introduction
#### The ***2023* Formula 1 season** represented the full optimisation phase of the ground-effect regulations and was defined by an **unprecedented level of dominance from Max Verstappen and Red Bull Racing**, who produced one of the most statistically overwhelming seasons in F1 history; Red Bull won **21 of 22 races**, with Verstappen securing his **third World Championship** in record-breaking fashion, while Ferrari and Mercedes traded sporadic podium contention amid inconsistent performance, tyre degradation challenges, and operational variability, and midfield teams experienced significant volatility under stable regulations, making 2023 a season characterised by **extreme competitive convergence at the front**, **regulatory maturity**, and **historical performance asymmetry**.

## Importing the necessary libraries

In [1]:
import pandas as pd
import fastf1
import logging

In [2]:
fastf1.Cache.enable_cache("../cache")

## Getting the Races of 2023

In [3]:
schedule = fastf1.get_event_schedule(2023)
listOfRaces = schedule["Country"].tolist()
listOfRaces

['Bahrain',
 'Bahrain',
 'Saudi Arabia',
 'Australia',
 'Azerbaijan',
 'United States',
 'Monaco',
 'Spain',
 'Canada',
 'Austria',
 'Great Britain',
 'Hungary',
 'Belgium',
 'Netherlands',
 'Italy',
 'Singapore',
 'Japan',
 'Qatar',
 'United States',
 'Mexico',
 'Brazil',
 'United States',
 'Abu Dhabi']

## Iterating into every Race and concatenating every Race Data into a DataFrame
Through my first try in ["Bahrain_test.ipynb"](https://github.com/Chracker24/MTS-IE/blob/main/02_Notebooks/Data_Collection/Formula1/2020/Bahrain_test.ipynb), I am going to use a loop to collect data and trim it down to the column and data points that I deem necessary for the Intelligence Engine.

In [4]:
cols = [
    "Driver",
    "LapTime",
    "LapNumber",
    "Stint",
    "Sector1Time",
    "Sector2Time",
    "Sector3Time",
    "Compound",
    "Team",
    "Deleted",
]

In [5]:
Season_Data2023 = []

In [9]:
logging.getLogger("fastf1").setLevel(logging.ERROR)
for i in range(len(listOfRaces)):
    session = fastf1.get_session(2023, listOfRaces[i], "R")
    session.load(laps=True, telemetry=False, weather=False)
    laps = session.laps
    Race_Data = laps[cols].copy()
    Race_Data = Race_Data.dropna(subset=["LapTime"]).copy()
    Race_Data["Season"] = 2023
    Race_Data["Race"] = listOfRaces[i]
    Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]] = Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]].apply(lambda x : x.dt.total_seconds())
    Race_Data = Race_Data[Race_Data["Deleted"]==False].copy()

    Season_Data2023.append(Race_Data)

In [7]:
season_2023 = pd.concat(Season_Data2023, ignore_index=True)

In [11]:
season_2023 = season_2023.sort_values("LapNumber").reset_index(drop=True)
season_2023

,Driver,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,Compound,Team,Deleted,Season,Race
0,TSU,95.096,1.0,1.0,NaN,36.255,22.864,MEDIUM,AlphaTauri,False,2023,Mexico
1,VER,99.019,1.0,1.0,NaN,42.414,23.842,SOFT,Red Bull Racing,False,2023,Bahrain
2,TSU,109.255,1.0,1.0,NaN,42.462,34.868,MEDIUM,AlphaTauri,False,2023,United States
3,PIA,111.927,1.0,1.0,NaN,46.249,28.945,MEDIUM,McLaren,False,2023,Singapore
4,GAS,108.429,1.0,1.0,NaN,41.792,35.191,MEDIUM,Alpine,False,2023,United States
...,...,...,...,...,...,...,...,...,...,...,...,...
24603,VER,85.201,78.0,2.0,22.353,40.652,22.196,INTERMEDIATE,Red Bull Racing,False,2023,Monaco
24604,HAM,85.105,78.0,3.0,22.304,40.622,22.179,INTERMEDIATE,Mercedes,False,2023,Monaco
24605,ALO,85.668,78.0,3.0,22.513,40.910,22.245,INTERMEDIATE,Aston Martin,False,2023,Monaco
24606,GAS,86.578,78.0,3.0,22.610,41.552,22.416,INTERMEDIATE,Alpine,False,2023,Monaco


## Adding the Dataframe as a CSV file in Data/Archive

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[3]
archive_dir = PROJECT_ROOT / "Data" / "Archive"

archive_dir.mkdir(parents=True, exist_ok=True)

season_2023.to_csv(
    archive_dir / "season_2023.csv",
    index=False
)